# **1. Matériel : vérifier que le GPU est bien sélectionné**

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Sun May 12 11:34:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

NameError: name 'torch' is not defined

# **2. Importation des librairies**

In [ ]:
import sys,os
import torch, torchvision
#import torchvision.transforms as transforms
from torchvision import datasets, transforms
import torch.nn as nn
from torchsummary import summary

from torch import Tensor
from functools import partial

from torchvision.transforms._presets import ImageClassification
import torchvision.models as models
from torchvision.models import ResNet50_Weights
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

# **3. Importation des données**

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
folder='Gdrive' #@param ['Gdrive']
if folder == 'Gdrive' :
  image_classification_path = '/content/drive/MyDrive/IA/UMONS/Workshop/2. Image classification'
  training_path=   image_classification_path + '/' + 'data'
sys.path.append(image_classification_path)
os.chdir(image_classification_path)


# **4. Paramètres**

In [ ]:
batch_size=16 #@param [16,32]
validation_split = 0.1
seed = 50 #@param
classifier='ResNet50v2' #@param ['ResNet50v2']
#base_model = models.resnet50(pretrained=True)

if classifier == 'ResNet50v2':
  weights=ResNet50_Weights.DEFAULT
  base_model=models.resnet50(weights=weights).to(device)
  preprocess = weights.transforms()




# **5. Visualiser une image**

In [ ]:
transformView = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()
                               ])
datasetView = datasets.ImageFolder(training_path,transformView)
dataloaderView = torch.utils.data.DataLoader(datasetView, batch_size=batch_size, shuffle=True)

In [ ]:
images,labels = next(iter(dataloaderView))
imshow(images[0].permute(1, 2, 0))

# **6. Visualiser une image préparée pour le modèle de base**

In [ ]:
#ancien

from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

dataset = datasets.ImageFolder(training_path,preprocess)

# generate indices: instead of the actual data we pass in integers instead
#train_indices, valid_indices, _, _ = train_test_split(
    #range(len(dataset)),
    #dataset.targets,
    #stratify=dataset.targets,
    #test_size=validation_split,
    #random_state=seed
#)

train_indices, valid_indices, _, _ = train_test_split(
    np.arange(dataset.targets),
    dataset.targets,
    stratify=dataset.targets,
    test_size=validation_split,
    random_state=seed
)

# generate subset based on indices
train_dataset = Subset(dataset, train_indices)
validation_dataset = Subset(dataset, valid_indices)

# create batches
dataloader={}
dataloader['train'] = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dataloader['validation']  = DataLoader(validation_dataset, batch_size=batch_size)

In [ ]:
dataset = datasets.ImageFolder(training_path,preprocess)
generator1 = torch.Generator().manual_seed(seed)
dataloader={}
dataset= torch.utils.data.random_split(dataset, [0.8,0.2],generator=generator1)
dataset={'train':dataset[0],'validation':dataset[1]}
dataloader = {'train':DataLoader(dataset=dataset['train'], shuffle=True, batch_size=batch_size, drop_last=True),
                'validation':DataLoader(dataset=dataset['validation'], shuffle=False, batch_size=batch_size)
}
print(len(dataloader['train']),len(dataloader['validation']))




In [ ]:
images,labels = next(iter(dataloader['train']))
print(images[0])
imshow(images[0].permute(1, 2, 0))
#imshow(images[0])
print(labels[0])

# **7. Construction du modèle**

In [ ]:
#ancien

class Model(nn.Module):
  def __init__(self):
      super().__init__()
      self.base_model = base_model
      for param in self.base_model.parameters():
        param.requires_grad = False
      self.fc = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3,inplace=True),
            nn.Linear(1024, 3),
            nn.Softmax()
            )

  def forward(self,x):
    self.to(device)
    x = self.fc(self.base_model(x))
    return x

model=Model()


In [ ]:
del model
model=base_model
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Sequential(
              nn.Linear(2048, 1024),
              nn.ReLU(inplace=True),
              nn.Dropout(p=0.3),
              nn.Linear(1024, 3),
              nn.Softmax(-1)
              )
model=model.to(device)
criterion = nn.CrossEntropyLoss()


# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [ ]:
summary(model, (3, 224, 224))

In [ ]:
print(model)

# **8. Entraînement du modèle**

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloader[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
              scheduler.step()

            epoch_loss = running_loss / len(dataset[phase])
            epoch_acc = running_corrects.double() / len(dataset[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [ ]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler,num_epochs=12)

# **Brouillon**

In [ ]:
#pour avoir la distribution des classes
print(dataset.class_to_idx)
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}
print(idx_to_class)
def get_class_distribution(dataset):
    count_dict = {k:0 for k,v in dataset.class_to_idx.items()} # initialise dictionary

    for input, label in dataset:
        label = idx_to_class[label]
        count_dict[label] += 1

    return count_dict

print("Distribution of classes: ", get_class_distribution(dataset))

In [ ]:
class Model(nn.Module):
  def __init__(self, backbone):
      super().__init__()
      self.pretrained_model = None
      self.classifier_layers = []
      self.new_layers = []

      self.pretrained_model = base_model
      self.classifier_layers = [self.pretrained_model.fc]
      self.pretrained_model.fc = nn.Linear(
          in_features=2048, out_features=3, bias=True
      )
      self.pretrained_model.fc = nn.Sequential(
          torch.nn.AvgPool2d(kernel_size = 100, stride = 0, padding = 0, ceil_mode=False, count_include_pad=True),
          nn.Linear(1024, 512),


      out_features=3, bias=True
      )
      x = torch.nn.AvgPool2d(kernel_size = 100, stride = 0, padding = 0, ceil_mode=False, count_include_pad=True)(x)
      self.new_layers = [self.pretrained_model.fc]

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

  def freezing(self):
    for p in self.pretrained_model:
      p.requires_grad = False
    for l in self.new_layers:
      for p in l.parameters():
          p.requires_grad = True

  def get_optimizer_params(self):
    """This method is used only during model fine-tuning when we need to
    set a linearly or exponentially decaying learning rate (LR) for the
    layers in the model. We exponentially decay the learning rate as we
    move away from the last output layer.
    """
    options = []
    # For the resnet class of models, we decay the LR exponentially and reduce
    # it to a third of the previous value at each step.
    layers = ["conv1", "bn1", "layer1", "layer2", "layer3", "layer4", "fc"]
    lr = 0.0001
    for layer_name in reversed(layers):
        options.append(
            {
                "params": getattr(self.pretrained_model, layer_name).parameters(),
                "lr": lr,
            }
        )
        lr = lr / 3.0
    # end for
    return options

In [ ]:
optimizer = torch.optim.Adam(fc.get_optimizer_params(), lr=1e-8)

In [ ]:
class Model(nn.Module):
  def __init__(self):
    self.conv1 = base_model

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
dataset = datasets.ImageFolder(training_path,preprocess)
dataloaders = {
    'train':
    torch.utils.data.DataLoader(dataset['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)  # for Kaggle
}
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

In [ ]:
next(iter(dataloader))

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')